<b>Шаг 1: Задача и варианты решения:</b>

Разработать классификатор для автоматического определения категории запроса по тексту сообщения, оставленному на сайте Администрации Курской области.

<b>Варианты решения:</b>

Будем использовать два метода: НЛП и машинное обучение.
<ul>
 <li>NLP для предварительной обработки текстовых данных в чистые и читаемые для машины;</li>
 <li>Мы используем машинное обучение для построения модели для обучения классификации входных текстовых данных.</li>
</ul>    

<b>Шаг 2: Обработка данных</b>

Разработать классификатор для автоматического определения категории запроса по тексту сообщения, оставленному на сайте Администрации Курской области.

In [38]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import VotingClassifier
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

In [39]:
sample_df = pd.read_csv('dataset/sample_solution.csv')
train_df = pd.read_csv('dataset/train_dataset_train.csv')
test_df = pd.read_csv('dataset/test_dataset_test.csv')

print(f"Training data shape:{sample_df.shape}")
print(f"Training data shape:{train_df.shape}")
print(f"Test data shape:{test_df.shape}")

Training data shape:(1000, 2)
Training data shape:(831, 11)
Test data shape:(1000, 4)


In [40]:
sample_df.head()

,id,Категория
0,843,0
1,1422,0
2,2782,0
3,2704,0
4,1,0


In [41]:
train_df.head()

,зины “Пятерочка” &nbsp;и “Усадьба” &nbsp;(адрес здания ул. К. Маркса 77/2). Позднее на заезде на парковку данного торгового центра был установлен сфетофор,который усугубит и без того сложную транспортную ситуацию в данном районе. Если раньше пробка из центра в сторону проспекта Дериглазова стояла до сфетофор на повороте на ТЦ Метро,то теперь она продолжается после данного светофора до поворота на п. Искра. Дело в том,что светофор на ТЦ Усадьба установлен в 100 метрах от светофора на повороте на п. Искра и два подряд установленных светофора блокируют движение. Функционально светофор на ТЦ Усадьба регулирует пешеходный переход на другую сторону дороги,вот только там никто не ходит,и выезд от ТЦ на противоположную сторону движения в сторону города - 0-1 автомобиль за один сеанс светофора,но весь поток при этом стоит. Слишком дорогая цена за возможность левого поворота. Разворот можно сделать на следующем светофоре или напротив ТЦ Европа. Опять создали условия для бизнеса,"но при этом создали новую пробку. Демонтаж данного светофора поможет хотя бы немного улучшить ситуацию с пробками на проспекте Дериглазова.&nbsp;</p>""",Светофоры на дорогах в границах городских округов и сельских поселений,Комитет дорожного хозяйства города Курска,0
0,2587,Сухое дерево на `Ермошкино озеро`,Аварийные деревья,Комитет городского хозяйства города Курска,3,NaN,NaN,NaN,NaN,NaN,NaN
1,1233,"<p>Добрый день, Роман Владимирович! Хочу обрат...",Меры поддержки в условиях неблагоприятной эпид...,Комитет информации и печати Курской области,6,NaN,NaN,NaN,NaN,NaN,NaN
2,272,<p>Просадка решётки дождеприёмника ливневой ка...,Нарушение дорожного покрытия (ямы) на дорогах...,Администрация города Курска,0,NaN,NaN,NaN,NaN,NaN,NaN
3,2107,Опора электросетей находится в аварийном состо...,"Нарушения, связанные с содержанием электросети...",Администрация города Курска,3,NaN,NaN,NaN,NaN,NaN,NaN
4,342,<p>Добрый день!</p><p>По улице 1 Суворовский п...,Некачественное водоснабжение в районе частного...,Администрация города Курска,3,NaN,NaN,NaN,NaN,NaN,NaN


In [42]:
test_df.head()

,id,Текст Сообщения,Тематика,Ответственное лицо
0,843,<p>Здравствуйте. На улице Мира &nbsp;было заме...,Неработающее наружное освещение,Администрация Курчатовского района
1,1422,<p>Уже вторую неделю не горит уличное освещени...,Неработающее наружное освещение,Комитет жилищно-коммунального хозяйства города...
2,2782,Не работает освещение во дворе дома 11а по Эне...,Неработающее наружное освещение,Комитет жилищно-коммунального хозяйства города...
3,2704,После покоса сорной растительности на газоне м...,Неудовлетворительная уборка улиц и тротуаров,Администрация Центрального округа города Курска
4,1,<p>Прошу принять меры к водителю маршрута 263:...,Неудовлетворительный внешний вид (поведение) в...,Администрация города Курска


In [43]:
test_df['Тематика'].unique()

array(['Неработающее наружное освещение',
       'Неудовлетворительная уборка улиц и тротуаров',
       'Неудовлетворительный внешний вид (поведение) водителя при осуществлении муниципальных (пригородных) перевозок',
       'Необходимо назначение дополнительных поездов',
       'Не соответствующий установленным нормам сбор и вывоз твёрдых бытовых отходов в районах многоквартирных домов',
       'Нарушение теплоснабжения многоквартирного дома',
       'Нарушения в вопросах оплаты услуг ЖКХ',
       'Неудовлетворительное материально-техническое состояние учреждений здравоохранения',
       'Наличие очереди на места в дошкольные учреждения',
       'Длительное неисполнение заявок управляющей компанией',
       'Несанкционированные свалки твёрдых бытовых отходов',
       'Очистка от снега и наледи дорог в границах городских округов и сельских поселений',
       'Неудовлетворительное санитарное состояние транспорта при осуществлении  муниципальных (пригородных) перевозок',
       'Нарушение

In [44]:
test_df['Тематика'].value_counts()

Нарушение дорожного покрытия (ямы)  на дорогах в границах городских округов и сельских поселений    77
Неработающее наружное освещение                                                                     54
Аварийные деревья                                                                                   54
Неудовлетворительная уборка улиц и тротуаров                                                        38
Несанкционированные свалки твёрдых бытовых отходов                                                  38
                                                                                                    ..
Проблемы с записью на вакцинацию                                                                     1
Несанкционированный сброс жидких бытовых отходов                                                     1
Нарушения в вопросах оплаты коммунальных услуг                                                       1
Несвоевременная выплата заработной платы                                 

In [45]:
test_df['Ответственное лицо'].unique()

array(['Администрация Курчатовского района',
       'Комитет жилищно-коммунального хозяйства города Курска',
       'Администрация Центрального округа города Курска',
       'Администрация города Курска', 'Администрация Льговского района',
       'ООО `УК Курский завод КПД им. А.Ф. Дериглазова`',
       'Государственная жилищная инспекция Курской области',
       'Комитет здравоохранения Курской области',
       'Администрация города Железногорска',
       'Комитет дорожного хозяйства города Курска',
       'Администрация Кореневского района',
       'Администрация Медвенского района',
       'Муниципальное казенное учреждение «Городская инспекция по жилищно-коммунальному хозяйству и благоустройству»',
       'Комитет социальной защиты населения города Курска',
       'Комитет природных ресурсов Курской области',
       'Комитет транспорта и автомобильных дорог Курской области',
       'Комитет социального обеспечения, материнства и детства Курской области',
       'Комитет городского 

In [46]:
test_df['Ответственное лицо'].value_counts()

Администрация города Курска                                     199
Комитет дорожного хозяйства города Курска                       138
Государственная жилищная инспекция Курской области               80
Комитет жилищно-коммунального хозяйства города Курска            76
Комитет городского хозяйства города Курска                       73
                                                               ... 
Комитет финансово-бюджетного контроля Курской области             1
Администрация Горшеченского района                                1
Комитет по управлению муниципальным имуществом города Курска      1
Администрация Рыльского района                                    1
ООО `УК `Маяк`                                                    1
Name: Ответственное лицо, Length: 64, dtype: int64